# Module 2 Exercise

In this exercise, you will create a Shiny app similar to the example in the ```L2_Practice_Layout``` notebook. Here is a cheatsheet for Shiny: [Shiny cheatsheet](http://shiny.rstudio.com/images/shiny-cheatsheet.pdf).

Keep a copy of your app.R in this exercise notebook; save it in a code cell, it'll be easier for you to match parentheses of functions.


You will use the **gapminder data** to create your interactive visualization. Make sure you do all the necessary data preprocessing only once outside the UI and server code. Following are pieces of sample code that you will need to modify and use to create the visualization in the Shiny app. Study them first. 

In [ ]:
library(ggplot2)
library(dplyr)
#gapminder <- readRDS(file = "/dsa/data/all_datasets/gapminder_data.rds")
gapminder <- readRDS(file = "/dsa/data/all_datasets/gapminder_data.rds")

gm <- gapminder 
gm <- transform(gm, pop = as.numeric(pop))
head(gm)

In [ ]:
# Code to plot life expectancy for a given year (all countries). 

ggplot(data=subset(gm, year==1952), aes(x=lifeExp)) +  geom_density(alpha=.2, fill="red") + xlab("Life Expectancy")

In [ ]:
# Code to aggregate data per continent
# compute total pop and weighted means for life exp. gdp per cap. 
cgm <- gm %>% group_by(continent,year) %>% summarise(totpop=sum(pop),avglifeExp=sum(pop*lifeExp)/totpop,avggdpPercap=sum(pop*gdpPercap)/totpop, numCountries=n())

head(cgm)

In [ ]:
# Code to plot total population per continent for a given year 

ggplot(data=subset(cgm,year==1952), aes(x=continent, y=totpop, fill=continent)) + geom_bar(stat="identity") 

In [ ]:
# Code to plot average GPD per capita per continent for a given year

ggplot(data=subset(cgm,year==1952), aes(x=continent, y=avggdpPercap, fill=continent)) + geom_bar(stat="identity") 


**Exercise 1: ** 

- Given these pieces of code, create a Shiny app that displays the plots for life expectancy for all countries, population per continent, and the average GDP per capita per continent **for a given year.**

- **Year** will be a ** *slider* input** between 1952 and 2007 with 5 year steps chosen by the user.

- ** Create three tabs ** with the plots in one tab, and gm and cgm data tables in the other two tabs. 

- Make sure you create the app.R file and save it to the folder named **`module2-exercise1`** under your **```exercises```** folder, and deploy it by running the following code cell.   


Design a clean look with nice background, no clutter, using appropriate captions, font sizes, styles, etc. You can design the dashboard in any way you want. 

PS: Pay attention to variable names, input output names, and the ggplot aes_string.  


In [ ]:
#DEPLOY CODE for your exercises
dir <- getwd() #This gets the current Working Directory
course <- "DATA-SCI-8654" #This is to specify the course path for the shiny server
folder <- "module2-exercise1" #This specifies the folder name to copy
#folder <- "module1-exercise2" #This specifies the folder name to copy

system(sprintf("/usr/local/bin/shiny_deploy %s %s %s", course, dir,folder), 
       intern = TRUE,
       ignore.stdout = FALSE, 
       ignore.stderr = FALSE,
       wait = TRUE, 
       input = NULL)


### Your code should produce the following at the minimum; you are expected to do a better design than this: 

<img src="../images/exercise_app.png">



In [ ]:
# Do not run this code cell, copy and paste it to app.R

library(ggplot2)
library(dplyr)
#gapminder <- readRDS(file = "/dsa/data/all_datasets/gapminder_data.rds")
gapminder <- readRDS(file = "/dsa/data/all_datasets/gapminder_data.rds")

# these operations need to be done only once

gm <- gapminder 
gm <- transform(gm, pop = as.numeric(pop))
# compute total pop and weighted means for life exp. gdp per cap. 
cgm <- gm %>% group_by(continent,year) %>% summarise(totpop=sum(pop),avglifeExp=sum(pop*lifeExp)/totpop,avggdpPercap=sum(pop*gdpPercap)/totpop, numCountries=n())


pop_ui <- fluidPage(
    
  h1("GapMinder"),
  tabsetPanel(    
   tabPanel("Plots",     

    fluidRow(      
        column(4, plotOutput("plot1")),
        column(4, plotOutput("plot2")),
        column(4, plotOutput("plot3"))
    ),
    
    hr(),
    
    fluidRow(
        column(4,
              helpText("GapMinder data: worlwide average life expectancy and GDP per capita 1952 - 2007.")
        ),
        column(8,
                sliderInput("year_in", 
                  label = em("Year:"), 1952, 2007, 1952, 5, width="100%", sep="")
                  
        )
    )             
   ),
            
   tabPanel("Country Data", dataTableOutput("table1")), 
   tabPanel("Continent Data", dataTableOutput("table2"))       
  )
)
    

pop_server <- function(input, output) {

         
  output$plot1 <- renderPlot({ 
      
      p <- ggplot(data=subset(gm, year==input$year_in), aes(x=lifeExp)) +  geom_density(alpha=.2, fill="red")             
      p <- p + theme_minimal() + xlab("Years") + ylab("Life Expectancy") + xlim(0,100) 
      print(p)
  })
 
  output$plot2 <- renderPlot({ 
      
      p <- ggplot(data=subset(cgm,year==input$year_in), aes(x=continent, y=totpop, fill=continent)) + geom_bar(stat="identity")       
      p <- p + theme_minimal() + xlab("Continents") + ylab("Total Population") + theme(legend.position="none") + ylim(0,4000000000) 
      print(p)
  })

    output$plot3 <- renderPlot({ 
      
      p <- ggplot(data=subset(cgm,year==input$year_in), aes(x=continent, y=avggdpPercap, fill=continent)) + geom_bar(stat="identity")      
      p <- p + theme_minimal() + xlab("Continents") + ylab("GDP per Capita") + theme(legend.position="none") + ylim(0,40000) 
      print(p)
  })    
    
  output$table1 <- renderDataTable(gm)    
  output$table2 <- renderDataTable(cgm)    
    
}

shinyApp(ui=pop_ui, server=pop_server)